# Group Project Report: Diagnostic Model for Chest Pain

### Section 009 Group 3 (Adeesh Devanand, Eddie Han, Connor Law, Burak Ozkan)

## 1) Introduction

Heart disease is an umbrella term used to define a range of health conditions that **negatively affect** the health of **the heart** or more generally the cardiovascular system, such as **coronary artery disease** which is the narrowing of blood vessels by cholesterol deposits that, especially when it occurs in coronary arteries, **can lead to a heart attack** and sudden death (CDC, 2021). **Chest pain** is a symptom of a heart attack and indicates to the patient that they should seek immediate primary care (NHS, 2023). 

For our project, we would like to explore the following question: **can variables measured during the diagnosis of a heart disease be used to predict the type of chest pain a patient with heart disease might experience during a future heart attack?** We are using the "**Heart Disease Dataset**" by David Lapp, obtained from the Kaggle website (https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset/data) which contains data regarding patients' health predictors, specifically acquired from the **Cleveland data set**, for our model.

## 2) Methods & Results

#### **a) First, we minimize the number of rows for the tables and call the libraries.**

In [5]:
options(repr.matrix.max.rows = 6)
library(repr)
library(tidyverse)
library(tidymodels)
library(RColorBrewer)
library(rvest)
install.packages("kknn")
library("kknn")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



#### **b) Secondly, we import the data from the heart.csv file that we have uploaded to our GitHub server by looking at the raw data and inputting the link into the read_csv function which yields the data.**

In [6]:
url <- read_csv("https://raw.githubusercontent.com/BurakMOzkan/DSCI100_group_3/main/heart.csv?token=GHSAT0AAAAAACKF3K3PIG64BRMTXAP2L4EEZKTB5CA")
url

Rows: 1025 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (14): age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpea...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
50,0,0,110,254,0,0,159,0,0.0,2,0,2,1
54,1,0,120,188,0,1,113,0,1.4,1,1,3,0


#### **c) We then wrangle the data by making the response variable of factor data type, changing its name, selecting for the required variables and renaming the rest of the columns to render them comprehensible without additional metadata.**

In [7]:
heart_disease <- url |>
                 mutate(chest_pain_type = as_factor(cp)) |>
                 select(age, trestbps, chol, thalach, chest_pain_type) |>
                 rename(resting_bp = trestbps,
                        serum_cholesterol = chol,
                        max_heart_rate = thalach) 
heart_disease

age,resting_bp,serum_cholesterol,max_heart_rate,chest_pain_type
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
52,125,212,168,0
53,140,203,155,0
70,145,174,125,0
⋮,⋮,⋮,⋮,⋮
47,110,275,118,0
50,110,254,159,0
54,120,188,113,0


#### **d) Before we build our model, divide the data into training and testing data sets, as well as set the seed to make the data split reproducible.**

In [8]:
set.seed(1)
hd_split <- initial_split(heart_disease, prop = 0.80, strata = chest_pain_type)
hd_train <- training(hd_split)
hd_test <- testing(hd_split) 

#### **e) Before we train the classifier, we should preprocess the data as it is sensitive to the scale of the predictors.**

In [9]:
hd_recipe <- recipe(chest_pain_type ~ ., data = hd_train) |>
             step_scale(all_predictors()) |>
             step_center(all_predictors())

#### **f) We can now train the classifier by randomly using K = 4.**

In [11]:
hd_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 4) |>
           set_engine("kknn") |>
           set_mode("classification")

hd_fit <- workflow() |>
          add_recipe(hd_recipe) |>
          add_model(hd_spec) |>
          fit(data = hd_train)

#### **g) Adding our predictions column to the data set.**

In [12]:
hd_predictions <- predict(hd_fit, hd_test) |>
                  bind_cols(hd_test)
hd_predictions

.pred_class,age,resting_bp,serum_cholesterol,max_heart_rate,chest_pain_type
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
0,46,120,249,144,0
0,71,112,149,125,0
0,43,132,341,136,0
⋮,⋮,⋮,⋮,⋮,⋮
2,58,140,211,165,2
3,65,138,282,174,3
1,41,110,172,158,0


#### **h) Let's assess the performance of the classifier and see if we need to choose a different K value.**

In [13]:
hd_accuracy <- hd_predictions |>
               metrics(truth = chest_pain_type, estimate = .pred_class) |>
               filter(.metric == "accuracy")
hd_accuracy

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,multiclass,0.7391304


**The accuracy is not great at all for K = 4. We need to further evaluate our model to increase its accuracy if we hope to use it in clinical applications.**

#### **i) Employing cross-validation, we can determine the best value for K.**

In [ ]:
hd_vfold <- vfold_cv(hd_train, v = 5, strata = chest_pain_type)

hd_spec_2 <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("classification")

hd_k_vals <- tibble(neighbors = seq(from = 1, to = 100, by = 5))

hd_results <- workflow() |>
              add_recipe(hd_recipe) |>
              add_model(hd_spec_2) |>
              tune_grid(resamples = hd_vfold, grid = hd_k_vals) |>
              collect_metrics() 

hd_accuracies <- hd_results |>
                filter(.metric == "accuracy")
hd_accuracies

#### **j) Plot the accuracies to pick a value for K.**

In [ ]:
which_k <- hd_accuracies |>
           ggplot(aes(x = neighbors, y = mean)) +
           geom_point() +
           geom_line() +
           labs(x = "Neighbors", y = "Accuracy") + 
           theme(text = element_text(size = 12))
which_k